In [ ]:
# input :
# 1) Judge validation name JSON file (specific to the court)
# 2) Bench validation names JSON file (specific to the court)
# 3) Judges' ids JSON file (specific to the court) -- for Judge and bench

# output : updated json files with correct names and ids

In [3]:
import json
import pandas as pd
import numpy as np
import path 
import glob
import os.path
from pathlib import Path
from tqdm import tqdm
import datetime
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [4]:
def read_json(json_file):  
    with open(Path(json_file),"r") as input_file:
        json_case = json.load(input_file)
        
    return json_case 

In [5]:
metadata_path = '/Users/arya/Desktop/goodBadJudges' # change path here 

with open(metadata_path+"/"+"/"+"judgelink_vdate.json", "r") as outfile:
    judgeinfo = json.load(outfile)
    
with open(metadata_path+"/"+"/"+"benchlink_vdate.json", "r") as outfile:
    benchinfo = json.load(outfile)
    
# with open(metadata_path+"/"+"/"+"goodjudges_punjab_haryana.json", "r") as outfile:
#     gdjudges = json.load(outfile)
    
with open(metadata_path+"/"+"/"+"punjab_haryana.json", "r") as outfile:
    judge_ids = json.load(outfile)

In [8]:
path ='/Users/arya/Desktop/goodBadJudges/data' # change path here 

emptycount = 0
texts = []
for court in os.listdir(path):
    if not court.endswith(".zip"):
        if court == "Supreme Court": # change path here: can be changed to High court data as well
            for year in os.listdir(path+"/"+court):
                if year == ".DS_Store":
                    continue
                if int(year) >= int("1947"): # modified from if int(year)>=int("2000")  to if int(year)>=int("1800")
                    for month in os.listdir(path+"/"+court+"/"+year):
                        if month == ".DS_Store":
                            continue
                        for case in os.listdir(path+"/"+court+"/"+year+"/"+month):
                            if case.endswith(".json"):
                                with open(path+"/"+court+"/"+year+"/"+month+"/"+case,"r",encoding="utf-8") as o:
                                    caseinfo = json.load(o)

                                # update judge name and ID    
                                if caseinfo['judge'][0]['name'] != 'JUDGMENT:' or caseinfo['judge'][0]['name'] != 'JUDGEMENT:' or caseinfo['judge'][0]['name'] != '0':                                   
                                    
                                    # if judge name exists
                                    judge_name = caseinfo['judge'][0]['name'].strip()
                                    ind = judge_name.find(",")
                                    judge_name = judge_name[:ind]
                                    
                                    if judge_name in judgeinfo.keys():
                                        for j in judgeinfo[judge_name]:
                                            caseinfo['judge'][0]['name'] = j
                                            caseinfo['judge'][0]['id'] = judge_ids.get(j)

                                # update bench name and ID
                                if caseinfo['bench'] != '0':                                    
                                    for bench in caseinfo['bench']:
                                        if bench['id']== '0':
                                            for k,v in benchinfo.items():
                                                if k == "Empty":
                                                    continue
                                                if bench['name'].strip() in k:
                                                    bench['name'] = v[0]
                                                    bench['id'] = judge_ids.get(v[0])
                                                # if bench name is already right    
                                                elif(bench['name'].strip() in judge_ids.keys()):
                                                    bench['id'] = judge_ids.get(bench['name'].strip())
                               
                                    
                                with open(path+"/"+court+"/"+year+"/"+month+"/"+case, "w") as outfile:
                                    json.dump(caseinfo, outfile, indent=4)
                                    